In [34]:
# Import một số thư viện cần thiết.
import random
import numpy as np
import matplotlib.pyplot as plt
import keras

# Sử dụng một mẹo nhỏ để vẽ hình trên cùng một dòng thay vì mở cửa sổ mới
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # đặt kích thước mặc định cho hình
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Một mẹo nhỏ để notebook tự load lại các module bên ngoài;
# xem thêm tại http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# Set super parameters 

pca_retain = 800
batchsize = 100
zae_threshold = 1.0

 
pca_retain = 800
hid_layer_sizes = [4000, 1000, 4000]
batchsize = 100
zae_threshold = 1.0
momentum = 0.9
pretrain, zae:       lr = 0.001000, epc = 800
pretrain, lin:       lr = 0.000100, epc = 800, wd = 1.000
logistic regression: lr = 0.500000, epc = 1000
finetune:            lr = 0.005000, epc = 1000


In [38]:
# Load data
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Reshape
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))


# Add Neurobricks Package path
import sys
sys.path.append(r'/home/lntk/Desktop/cifar10/NeuroBricks/neurobricks')

# Preprocessing
from preprocess import SubtractMeanAndNormalizeH, PCA
import theano

print "\n... pre-processing"
preprocess_model = SubtractMeanAndNormalizeH(X_train.shape[1])
map_fun = theano.function([preprocess_model.varin], preprocess_model.output())

pca_obj = PCA()
pca_obj.fit(map_fun(X_train), retain=pca_retain, whiten=True)
preprocess_model = preprocess_model + pca_obj.forward_layer
preprocess_function = theano.function([preprocess_model.varin], preprocess_model.output())

pcamapping = theano.function([pca_obj.forward_layer.varin], pca_obj.forward_layer.output())
pcaback = theano.function([pca_obj.backward_layer.varin], pca_obj.backward_layer.output())

X_train = preprocess_function(X_train)
X_test = preprocess_function(X_test)

print "Done."


... pre-processing
Centralizing data...  Done.
Computing covariance...  Done.
Eigen-decomposition... Done. Maximum stable PCs: 3072
Number of selected PCs: 800, ratio of retained variance: 0.989758
Done.


In [39]:
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

('Training data shape: ', (50000, 800))
('Training labels shape: ', (50000, 1))
('Test data shape: ', (10000, 800))
('Test labels shape: ', (10000, 1))


In [44]:
# Change labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(4000, input_dim=800)) 
model.add(keras.layers.ThresholdedReLU(theta=zae_threshold))
model.add(Dense(1000))
model.add(Dense(4000))
model.add(keras.layers.ThresholdedReLU(theta=zae_threshold))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=100,
          batch_size=100)


Epoch 1/100
50000/50000 [==============================] - 61s 1ms/step - loss: 2.3030 - acc: 0.0991
Epoch 2/100
50000/50000 [==============================] - 62s 1ms/step - loss: 2.3020 - acc: 0.0958
Epoch 3/100
50000/50000 [==============================] - 62s 1ms/step - loss: 2.3010 - acc: 0.1053
Epoch 4/100
50000/50000 [==============================] - 61s 1ms/step - loss: 2.3001 - acc: 0.1059
Epoch 5/100
50000/50000 [==============================] - 64s 1ms/step - loss: 2.2992 - acc: 0.1100
Epoch 6/100
50000/50000 [==============================] - 62s 1ms/step - loss: 2.2984 - acc: 0.1117
Epoch 7/100
50000/50000 [==============================] - 62s 1ms/step - loss: 2.2975 - acc: 0.1117
Epoch 8/100
50000/50000 [==============================] - 62s 1ms/step - loss: 2.2966 - acc: 0.1123
Epoch 9/100
50000/50000 [==============================] - 62s 1ms/step - loss: 2.2958 - acc: 0.1122
Epoch 10/100
50000/50000 [==============================] - 64s 1ms/step - loss: 2.2949 - a

In [46]:
score = model.evaluate(X_test, y_test, batch_size=100)
print(score)

10000/10000 [==============================] - 3s 339us/step
[2.2171088075637817, 0.17220000095665455]


In [49]:
model.save('/home/lntk/Desktop/cifar10_1904_model.h5')